In [2]:
import glob
import time
import os
import pandas as pd
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler
import pickle
from argparse import ArgumentParser, Namespace
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from itertools import chain
from tqdm import tqdm
import copy
import shutil
import pickle

In [3]:
es_url = 'http://ckg07:9200'
es_index = 'wikidatadwd-augmented-08'

# Input Paths

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-train-canonical/
train_path = "/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/train1-canonical"

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-dev-canonical/
dev_path = "/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/dev-canonical"

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/ground_truth/Xinting_GT_csv
ground_truth_files = "/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/round4_gt_with_labels"


# OUTPUT PATHS
output_path = "/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/table-linker"
train_output_path = f'{output_path}/train1-output/tbl-pipeline-nn'
dev_output_path = f'{output_path}/dev-output/tbl-pipeline-nn'

# increase version to create a new folder for an experiment
PREVIOUS_VERSION = "v25"
VERSION = "v26"

train_candidate_path = f'{train_output_path}/{VERSION}/candidates'
train_trigram_candidate_path = f'{train_output_path}/{VERSION}/trigram_candidates'
train_features1_path = f'{train_output_path}/{VERSION}/features1'
train_trigram_features1_path = f'{train_output_path}/{VERSION}/trigram_features1'
train_features1_joined_path = f'{train_output_path}/{VERSION}/features1_joined'
train_features2_path = f'{train_output_path}/{VERSION}/features2'
train_features3_path = f'{train_output_path}/{VERSION}/features3'
train_feature_path = f'{train_output_path}/{VERSION}/features'
train_missing_candidates_path = f'{train_output_path}/{VERSION}/train_missing_candidates_path'

dev_candidate_path = f'{dev_output_path}/{VERSION}/candidates'
dev_trigram_candidate_path = f'{dev_output_path}/{VERSION}/trigram_candidates'
dev_feature_path = f'{dev_output_path}/{VERSION}/features'
dev_features1_path = f'{dev_output_path}/{VERSION}/features1'
dev_trigram_features1_path = f'{dev_output_path}/{VERSION}/trigram_features1'
dev_features1_joined_path = f'{dev_output_path}/{VERSION}/features1_joined'
dev_features2_path = f'{dev_output_path}/{VERSION}/features2'
dev_features3_path = f'{dev_output_path}/{VERSION}/features3'
dev_output_predictions = f'{dev_output_path}/{VERSION}/dev_predictions'
dev_predictions_top_k = f'{dev_output_path}/{VERSION}/dev_predictions_top_k'
dev_colorized_path = f'{dev_output_path}/{VERSION}/dev_predictions_colorized'
dev_metrics_path = f'{dev_output_path}/{VERSION}/dev_predictions_metrics'
dev_missing_candidates_path = f'{dev_output_path}/{VERSION}/dev_missing_candidates_path'

aux_field = 'graph_embedding_complex,class_count,property_count,context'


train_prop_count = f'{train_output_path}/{VERSION}/train_prop_count' 
train_trigram_prop_count = f'{train_output_path}/{VERSION}/train_trigram_prop_count'
train_joined_prop_count = f'{train_output_path}/{VERSION}/train_joined_prop_count'

train_class_count = f'{train_output_path}/{VERSION}/train_class_count'
train_trigram_class_count = f'{train_output_path}/{VERSION}/train_trigram_class_count'
train_joined_class_count = f'{train_output_path}/{VERSION}/train_joined_class_count'

train_context_path = f'{train_output_path}/{VERSION}/train_context'
train_trigram_context_path = f'{train_output_path}/{VERSION}/train_trigram_context'
train_joined_context_path = f'{train_output_path}/{VERSION}/train_joined_context'

train_graph_embedding = f'{train_output_path}/{VERSION}/train_graph_embedding'
train_trigram_graph_embedding = f'{train_output_path}/{VERSION}/train_trigram_graph_embedding'
train_joined_graph_embedding = f'{train_output_path}/{VERSION}/train_joined_graph_embedding'

train_context_property_path = f'{train_output_path}/{VERSION}/train_context_properties'

dev_prop_count = f'{dev_output_path}/{VERSION}/dev_prop_count'
dev_trigram_prop_count = f'{dev_output_path}/{VERSION}/dev_trigram_prop_count'
dev_joined_prop_count = f'{dev_output_path}/{VERSION}/dev_joined_prop_count'

dev_class_count = f'{dev_output_path}/{VERSION}/dev_class_count'
dev_trigram_class_count = f'{dev_output_path}/{VERSION}/dev_trigram_class_count'
dev_joined_class_count = f'{dev_output_path}/{VERSION}/dev_joined_class_count'

dev_context_path = f'{dev_output_path}/{VERSION}/dev_context'
dev_trigram_context_path = f'{dev_output_path}/{VERSION}/dev_trigram_context'
dev_joined_context_path = f'{dev_output_path}/{VERSION}/dev_joined_context'

dev_graph_embedding = f'{dev_output_path}/{VERSION}/dev_graph_embedding'
dev_trigram_graph_embedding = f'{dev_output_path}/{VERSION}/dev_trigram_graph_embedding'
dev_joined_graph_embedding = f'{dev_output_path}/{VERSION}/dev_joined_graph_embedding'

dev_context_property_path = f'{dev_output_path}/{VERSION}/dev_context_properties'
dev_incorrect_answers_path = f'{dev_output_path}/{VERSION}/incorrect_answers.csv'

temp_dir = f'{output_path}/temp'

pseudo_gt_model = '/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/table-linker/dev-output/pseudo-gt-nn/v19/saved_models/epoch_1_loss_0.534353494644165_top1_0.7883487007544007.pth'
pseudo_gt_min_max_scaler_path = '/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/table-linker/temp/training_data/normalization_factor.pkl'

tl_log_file =f'{temp_dir}/tl_log.txt'

training_data_path = f'{dev_output_path}/{VERSION}/training_data'
pos_output = f'{training_data_path}/tl_pipeline_pos_features.pkl'
neg_output = f'{training_data_path}/tl_pipeline_neg_features.pkl'
min_max_scaler_path = f'{training_data_path}/tl_pipeline_normalization_factor.pkl'

final_score_column = 'siamese_prediction'
threshold = final_score_column+":median"

model_save_path = f'{dev_output_path}/{VERSION}/saved_models'
best_model_path = ''

copy_candidates_from_previous_version = False
copy_features1_from_previous_version = False

In [ ]:
!mkdir -p "$temp_dir"

!mkdir -p "$train_trigram_candidate_path"
!mkdir -p "$train_trigram_features1_path"
!mkdir -p "$dev_trigram_candidate_path"
!mkdir -p "$dev_trigram_features1_path"

!mkdir -p "$train_prop_count"
!mkdir -p "$train_trigram_prop_count"
!mkdir -p "$dev_prop_count"
!mkdir -p "$dev_trigram_prop_count"
!mkdir -p "$train_class_count"
!mkdir -p "$train_trigram_class_count"
!mkdir -p "$dev_class_count"
!mkdir -p "$dev_trigram_class_count"
!mkdir -p "$train_graph_embedding"
!mkdir -p "$train_trigram_graph_embedding"
!mkdir -p "$dev_graph_embedding"
!mkdir -p "$dev_trigram_graph_embedding"
!mkdir -p "$train_context_path"
!mkdir -p "$train_trigram_context_path"
!mkdir -p "$dev_context_path"
!mkdir -p "$dev_trigram_context_path"

!mkdir -p "$train_candidate_path"
!mkdir -p "$dev_candidate_path"

!mkdir -p "$train_feature_path"
!mkdir -p "$train_features1_path"
!mkdir -p "$train_features2_path"
!mkdir -p "$train_features3_path"
!mkdir -p "$dev_feature_path"
!mkdir -p "$dev_features1_path"
!mkdir -p "$dev_features2_path"
!mkdir -p "$dev_features3_path"

!mkdir -p "$temp_dir/training_data"
!mkdir -p "$dev_output_predictions"
!mkdir -p "$model_save_path"
!mkdir -p "$dev_predictions_top_k"
!mkdir -p "$dev_colorized_path"
!mkdir -p "$dev_metrics_path"
!mkdir -p "$dev_missing_candidates_path"
!mkdir -p "$train_missing_candidates_path"
!mkdir -p "$train_context_property_path"
!mkdir -p "$dev_context_property_path"

!mkdir -p "$training_data_path"

!mkdir -p "$train_features1_joined_path"
!mkdir -p "$dev_features1_joined_path"
!mkdir -p "$train_joined_prop_count"
!mkdir -p "$train_joined_class_count"
!mkdir -p "$train_joined_context_path"
!mkdir -p "$train_joined_graph_embedding"
!mkdir -p "$dev_joined_prop_count"
!mkdir -p "$dev_joined_class_count"
!mkdir -p "$dev_joined_context_path"
!mkdir -p "$dev_joined_graph_embedding"

In [ ]:
if copy_candidates_from_previous_version:
    !cp $dev_output_path/$PREVIOUS_VERSION/candidates/*csv $dev_output_path/$VERSION/candidates
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_prop_count/* $dev_output_path/$VERSION/dev_prop_count
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_class_count/* $dev_output_path/$VERSION/dev_class_count
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_context/* $dev_output_path/$VERSION/dev_context
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_graph_embedding/* $dev_output_path/$VERSION/dev_graph_embedding
    
    !cp $train_output_path/$PREVIOUS_VERSION/candidates/*csv $train_output_path/$VERSION/candidates
    !cp $train_output_path/$PREVIOUS_VERSION/train_prop_count/* $train_output_path/$VERSION/train_prop_count
    !cp $train_output_path/$PREVIOUS_VERSION/train_class_count/* $train_output_path/$VERSION/train_class_count
    !cp $train_output_path/$PREVIOUS_VERSION/train_context/* $train_output_path/$VERSION/train_context
    !cp $train_output_path/$PREVIOUS_VERSION/train_graph_embedding/* $train_output_path/$VERSION/train_graph_embedding

In [4]:
pseudo_gt_features = ["monge_elkan","monge_elkan_aliases","jaro_winkler","levenshtein","singleton","pgr_rts","context_score","smc_class_score","smc_property_score"]

## Candidate Generation

In [ ]:
def candidate_generation(path, gt_path, output_path, class_count_path, prop_count_path, context_path, graph_embedding):
    file_list = glob.glob(path + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        st = time.time()
        filename = file.split('/')[-1]
        gt_file = f"{ground_truth_files}/{filename}"
        output_file = f"{output_path}/{filename}"
        
        !tl --log-file $tl_log_file clean -c label -o label_clean "$file" / \
        --url $es_url --index $es_index \
        get-fuzzy-augmented-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" / \
        --url $es_url --index $es_index \
        get-exact-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" / \
        / get-ngram-matches -c label_clean  \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" \
        / get-trigram-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" \
        / ground-truth-labeler --gt-file "$gt_file" > "$output_file"

        for field in aux_field.split(','):
            aux_list = []
            if field == 'context':
                file_list = glob.glob(f'{temp_dir}/*{field}.jl')
                o_f = open(f"{context_path}/{filename[:-4]}_context.jl", 'w')
                
                for i_f_P in file_list:
                    i_f = open(i_f_P)
                    for line in i_f:
                        o_f.write(line)
                    i_f.close()
                o_f.close()
                
            else:
                for f in glob.glob(f'{temp_dir}/*{field}.tsv'):
                    aux_list.append(pd.read_csv(f, sep='\t', dtype=object))
                aux_df = pd.concat(aux_list).drop_duplicates(subset=['qnode'])
                if field == 'class_count':
                    class_count_file = f"{class_count_path}/{filename[:-4]}_class_count.tsv"
                    aux_df.to_csv(class_count_file, sep='\t', index=False)
                elif field == 'property_count':
                    prop_count_file = f"{prop_count_path}/{filename[:-4]}_prop_count.tsv"
                    aux_df.to_csv(prop_count_file, sep='\t', index=False)
                else:
                    graph_embedding_file = f"{graph_embedding}/{filename[:-4 ]}_graph_embedding_complex.tsv"
                    aux_df.to_csv(graph_embedding_file, sep='\t', index=False)
        

In [ ]:
if not copy_candidates_from_previous_version:
    candidate_generation(train_path, ground_truth_files, train_candidate_path, train_class_count, train_prop_count, train_context_path,train_graph_embedding)

In [ ]:
if not copy_candidates_from_previous_version:
    candidate_generation(dev_path, ground_truth_files, dev_candidate_path, dev_class_count, dev_prop_count, dev_context_path, dev_graph_embedding)

## Feature Generation

In [5]:
def features_1(candidate_dir, output_path, context_path, class_count_dir, property_count_dir, pseudo_gt_model, pseudo_gt_min_max_scaler_path, context_property_path):
    file_list = glob.glob(candidate_dir + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        if os.path.getsize(file) == 0:
            continue
        filename = file.split('/')[-1]
        output_file = f"{output_path}/{filename}"
        
        context_file = f"{context_path}/{filename[:-4]}_context.jl"
        class_count_file = f"{class_count_dir}/{filename[:-4]}_class_count.tsv"
        property_count_file = f"{property_count_dir}/{filename[:-4]}_prop_count.tsv"
        feature_str =  ",".join(pseudo_gt_features)
        context_property_file = f"{context_property_path}/{filename[:-4]}_context_properties.csv"
        run = True
        if filename == '0Z8LKW3C.csv' or filename == '2OJXQ9F3.csv' or filename == '1QGWWYAF.csv':
            run=True
            
        if run:
            !tl --log-file $tl_log_file deduplicate-candidates -c kg_id $file \
                / string-similarity -i --method symmetric_monge_elkan:tokenizer=word -o monge_elkan --threshold 0.5 \
                / string-similarity -i --method symmetric_monge_elkan:tokenizer=word -c label_clean kg_aliases -o monge_elkan_aliases --threshold 0.5 \
                / string-similarity -i --method jaro_winkler -o jaro_winkler --threshold 0.5 \
                / string-similarity -i --method levenshtein -o levenshtein --threshold 0.5 \
                / create-singleton-feature -o singleton \
                / pick-hc-candidates -o ignore_candidate --string-similarity-label-columns monge_elkan,jaro_winkler,levenshtein --string-similarity-alias-columns monge_elkan_aliases \
                / context-match --debug --context-file $context_file --ignore-column-name ignore_candidate -o context_score \
                --similarity-string-threshold 0.85 --similarity-quantity-threshold 0.9 \
                --save-relevant-properties --context-properties-path $context_property_file \
                / kth-percentile -c context_score -o kth_percenter --ignore-column ignore_candidate --k-percentile 0.75  --minimum-cells 10 \
                / pgt-semantic-tf-idf \
                -o smc_class_score \
                --pagerank-column pagerank \
                --retrieval-score-column retrieval_score \
                --feature-file "$class_count_file" \
                --feature-name class_count \
                --high-confidence-column kth_percenter \
                / pgt-semantic-tf-idf \
                -o smc_property_score \
                --pagerank-column pagerank \
                --retrieval-score-column retrieval_score \
                --feature-file "$property_count_file" \
                --feature-name property_count \
                --high-confidence-column kth_percenter \
                / predict-using-model -o pseudo_gt_prediction \
                --features $feature_str \
                --ranking-model $pseudo_gt_model \
                --ignore-column ignore_candidate \
                --normalization-factor $pseudo_gt_min_max_scaler_path \
                / create-pseudo-gt -o pseudo_gt \
                --column-thresholds pseudo_gt_prediction:mean \
                --filter smc_class_score:0 > $output_file

In [6]:
if not copy_features1_from_previous_version:
    features_1(train_candidate_path, train_features1_path, train_context_path, train_class_count, train_prop_count,pseudo_gt_model, pseudo_gt_min_max_scaler_path, train_context_property_path)
else:
    !cp $train_output_path/$PREVIOUS_VERSION/features1/*csv $train_output_path/$VERSION/features1

0it [00:00, ?it/s]

/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


1it [00:21, 21.19s/it]

/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


2it [00:44, 22.43s/it]

...

299it [3:44:46, 69.00s/it]

/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


300it [3:45:06, 54.30s/it]

/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


301it [3:46:15, 45.10s/it]


In [7]:
if not copy_features1_from_previous_version:
    features_1(dev_candidate_path, dev_features1_path, dev_context_path, dev_class_count, dev_prop_count,pseudo_gt_model, pseudo_gt_min_max_scaler_path, dev_context_property_path)
else:
    !cp $dev_output_path/$PREVIOUS_VERSION/features1/*csv $dev_output_path/$VERSION/features1

0it [00:00, ?it/s]

/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


1it [00:49, 49.44s/it]

/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


2it [01:10, 33.00s/it]

...

49it [37:26, 52.50s/it]

/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


50it [37:46, 42.74s/it]

/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


51it [38:06, 44.83s/it]


In [ ]:
def ngram_candidate_generation(path, gt_path, output_path, class_count_path, prop_count_path, context_path, graph_embedding):
    file_list = glob.glob(path + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        filename = file.split('/')[-1]
        gt_file = f"{ground_truth_files}/{filename}"
        output_file = f"{output_path}/{filename}"
        
        os.path.getsize(file) == 0:
            continue
        
        !tl --log-file $tl_log_file --url $es_url --index $es_index \
        get-trigram-matches "$file" -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" \
        --pseudo-gt-column pseudo_gt \
        / ground-truth-labeler --gt-file "$gt_file" > "$output_file"

        for field in aux_field.split(','):
            aux_list = []
            for f in glob.glob(f'{temp_dir}/trigram_matches_*{field}.tsv'):
                aux_list.append(pd.read_csv(f, sep='\t', dtype=object))
            aux_df = pd.concat(aux_list).drop_duplicates(subset=['qnode'])
            if field == 'class_count':
                class_count_file = f"{class_count_path}/{filename.strip('.csv')}_class_count.tsv"
                aux_df.to_csv(class_count_file, sep='\t', index=False)
            elif field == 'property_count':
                prop_count_file = f"{prop_count_path}/{filename.strip('.csv')}_prop_count.tsv"
                aux_df.to_csv(prop_count_file, sep='\t', index=False)
            elif field == 'context':
                context_file = f"{context_path}/{filename.strip('.csv')}_context.tsv"
                aux_df.to_csv(context_file, sep='\t', index=False)
            else:
                graph_embedding_file = f"{graph_embedding}/{filename.strip('.csv')}_graph_embedding_complex.tsv"
                aux_df.to_csv(graph_embedding_file, sep='\t', index=False)
        

In [ ]:
ngram_candidate_generation(train_features1_path, ground_truth_files, train_trigram_candidate_path, train_trigram_class_count, train_trigram_prop_count, train_trigram_context_path, train_trigram_graph_embedding)

In [ ]:
ngram_candidate_generation(dev_features1_path, ground_truth_files, dev_trigram_candidate_path, dev_trigram_class_count, dev_trigram_prop_count, dev_trigram_context_path, dev_trigram_graph_embedding)

In [ ]:
features_1(train_trigram_candidate_path, train_trigram_features1_path, train_trigram_context_path, train_trigram_class_count, train_trigram_prop_count,pseudo_gt_model, pseudo_gt_min_max_scaler_path)getsize

In [ ]:
features_1(dev_trigram_candidate_path, dev_trigram_features1_path, dev_trigram_context_path, dev_trigram_class_count, dev_trigram_prop_count,pseudo_gt_model, pseudo_gt_min_max_scaler_path)

In [ ]:
def join_all_files(path1, path2, output_path, file_type='csv'):
    file_list = glob.glob(f'{path1}/*.{file_type}')
    sep='\t' if file_type == 'tsv' else ','
                          
    for i, file in tqdm(enumerate(file_list)):
         
        filename = file.split('/')[-1]
        output_file = f"{output_path}/{filename}"
        out = []
        if os.path.getsize(file) > 0:
            out.append(pd.read_csv(file, sep=sep))
        file2 = f"{path2}/{filename}"
        if os.path.exists(file2) and os.path.getsize(file2) > 0:
            out.append(pd.read_csv(file2, sep=sep))

        output_file = f"{output_path}/{filename}"
        out_df = pd.concat(out)
        if len(out_df) > 0:
            out_df.to_csv(output_file, index=False, sep=sep)
        
    

In [ ]:
join_all_files(train_prop_count,train_trigram_prop_count, train_joined_prop_count, file_type='tsv')

In [ ]:
join_all_files(train_class_count,train_trigram_class_count, train_joined_class_count, file_type='tsv')

In [ ]:
join_all_files(train_context_path,train_trigram_context_path, train_joined_context_path, file_type='tsv')

In [ ]:
join_all_files(train_graph_embedding,train_trigram_graph_embedding, train_joined_graph_embedding, file_type='tsv')

In [ ]:
join_all_files(dev_prop_count,dev_trigram_prop_count, dev_joined_prop_count, file_type='tsv')

In [ ]:
join_all_files(dev_class_count,dev_trigram_class_count, dev_joined_class_count, file_type='tsv')

In [ ]:
join_all_files(dev_context_path,dev_trigram_context_path, dev_joined_context_path, file_type='tsv')

In [ ]:
join_all_files(dev_graph_embedding,dev_trigram_graph_embedding, dev_joined_graph_embedding, file_type='tsv')

In [ ]:
join_all_files(train_features1_path, train_trigram_features1_path,train_features1_joined_path)

In [ ]:
join_all_files(dev_features1_path, dev_trigram_features1_path,dev_features1_joined_path)

In [8]:
def features_2(features1_dir, output_path, context_path, context_property_path):
    file_list = glob.glob(features1_dir + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        if os.path.getsize(file) == 0:
            continue
        filename = file.split('/')[-1]
        run = True
        if filename == '0Z8LKW3C.csv' or filename == '2OJXQ9F3.csv' or filename == '1QGWWYAF.csv':
            run=True
            
        output_file = f"{output_path}/{filename}"
        context_file = f"{context_path}/{filename[:-4]}_context.jl"
        context_property_file = f"{context_property_path}/{filename[:-4]}_context_properties.csv"
        if run:
            !tl --log-file $tl_log_file context-match $file --debug --context-file $context_file -o context_score_3 \
             --similarity-string-threshold 0.85 --similarity-quantity-threshold 0.9 \
             --use-relevant-properties --context-properties-path $context_property_file > $output_file

In [ ]:
features_2(train_features1_joined_path, train_features2_path, train_joined_context_path, train_context_property_path)

In [9]:
features_2(train_features1_path, train_features2_path, train_context_path, train_context_property_path)

295it [21:26,  4.36s/it]


In [ ]:
features_2(dev_features1_joined_path, dev_features2_path, dev_joined_context_path, dev_context_property_path)

In [10]:
features_2(dev_features1_path, dev_features2_path, dev_context_path, dev_context_property_path)

50it [03:26,  4.12s/it]


In [ ]:
def features_2_1(features2_dir, output_path, context_path, context_property_path):
    file_list = glob.glob(features2_dir + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        if os.path.getsize(file) == 0:
            continue
        
        filename = file.split('/')[-1]
        output_file = f"{output_path}/{filename}"
        context_file = f"{context_path}/{filename[:-4]}_context.tsv"
        context_property_file = f"{context_property_path}/{filename[:-4]}_context_properties.csv"
            
        if os.path.exists(output_file) and os.path.getsize(output_file) > 0:
            continue

        !tl context-match --log-file $tl_log_file --debug --context-file $context_file -o context_score_3 \
            --similarity-string-threshold 0.85 --similarity-quantity-threshold 0.9 \
            --use-relevant-properties --context-properties-path $context_property_file >$output_file

In [ ]:
features_2_1(train_features2_path, train_features3_path, train_joined_context_path, train_context_property_path)

In [ ]:
features_2_1(train_features2_path, train_features3_path, train_context_path, train_context_property_path)

In [ ]:
features_2_1(dev_features2_path, dev_features3_path, dev_joined_context_path, dev_context_property_path)

In [ ]:
features_2_1(dev_features2_path, dev_features3_path, dev_context_path, dev_context_property_path)

In [11]:
def features_3(features3_dir, output_path, ge_path, class_count_dir, property_count_dir):
    file_list = glob.glob(features3_dir + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        if os.path.getsize(file) == 0:
            continue
        filename = file.split('/')[-1]
        output_file = f"{output_path}/{filename}"
        embedding_file = f"{ge_path}/{filename[:-4]}_graph_embedding_complex.tsv"
        class_count_file = f"{class_count_dir}/{filename[:-4]}_class_count.tsv"
        property_count_file = f"{property_count_dir}/{filename[:-4]}_prop_count.tsv"

        !tl mosaic-features -c kg_labels --num-char --num-tokens $file \
        / score-using-embedding \
        --column-vector-strategy centroid-of-lof \
        --lof-strategy pseudo-gt \
        -o pgt_centroid_score \
        --embedding-file $embedding_file \
        / compute-tf-idf  \
        --feature-file $class_count_file \
        --feature-name class_count \
        --singleton-column pseudo_gt \
        -o pgt_class_count_tf_idf_score \
        / compute-tf-idf \
        --feature-file $property_count_file \
        --feature-name property_count \
        --singleton-column pseudo_gt \
        -o pgt_property_count_tf_idf_score > $output_file

In [ ]:
features_3(train_features3_path, train_feature_path, train_joined_graph_embedding, train_joined_class_count, train_joined_prop_count)

In [12]:
features_3(train_features2_path, train_feature_path, train_graph_embedding, train_class_count, train_prop_count)

0it [00:00, ?it/s]

mosaic-features Time: 0.005589008331298828s
Qnodes to lookup: 2345
Qnodes from file: 2322
Outlier removal generates 5 lof-voted candidates
score-using-embedding Time: 1.529102087020874s
compute-tf-idf-class_count Time: 2.0038797855377197s
compute-tf-idf-property_count Time: 2.2091290950775146s


1it [00:07,  7.03s/it]

mosaic-features Time: 0.012553930282592773s
Qnodes to lookup: 1507
Qnodes from file: 1485
Outlier removal generates 4 lof-voted candidates
score-using-embedding Time: 0.9691529273986816s
compute-tf-idf-class_count Time: 1.3930001258850098s
compute-tf-idf-property_count Time: 1.6014339923858643s


2it [00:13,  6.72s/it]

...

293it [39:37,  9.60s/it]

mosaic-features Time: 0.009004831314086914s
Qnodes to lookup: 2642
Qnodes from file: 2613
Outlier removal generates 2 lof-voted candidates
score-using-embedding Time: 1.0995862483978271s
compute-tf-idf-class_count Time: 1.5633111000061035s
compute-tf-idf-property_count Time: 1.83418607711792s


294it [39:44,  8.77s/it]

mosaic-features Time: 0.03783297538757324s
Qnodes to lookup: 3049
Qnodes from file: 3028
Outlier removal generates 3 lof-voted candidates
Outlier removal generates 2 lof-voted candidates
Column_vector_stragtegy centroid_of_lof failed
Outlier removal generates 2 lof-voted candidates
score-using-embedding Time: 2.114168167114258s
compute-tf-idf-class_count Time: 3.594047784805298s
compute-tf-idf-property_count Time: 4.220209836959839s


295it [39:54,  8.12s/it]


In [ ]:
features_3(dev_features3_path, dev_feature_path, dev_joined_graph_embedding, dev_joined_class_count, dev_joined_prop_count)

In [13]:
features_3(dev_features2_path, dev_feature_path, dev_graph_embedding, dev_class_count, dev_prop_count)

0it [00:00, ?it/s]

mosaic-features Time: 0.022246837615966797s
Qnodes to lookup: 4055
Qnodes from file: 3964
Outlier removal generates 2 lof-voted candidates
Column_vector_stragtegy centroid_of_lof failed
Column_vector_stragtegy centroid_of_lof failed
score-using-embedding Time: 1.7821359634399414s
compute-tf-idf-class_count Time: 2.6123058795928955s
compute-tf-idf-property_count Time: 3.130202054977417s


1it [00:08,  8.80s/it]

mosaic-features Time: 0.00925302505493164s
Qnodes to lookup: 1884
Qnodes from file: 1860
Outlier removal generates 4 lof-voted candidates
score-using-embedding Time: 0.987346887588501s
compute-tf-idf-class_count Time: 1.417754888534546s
compute-tf-idf-property_count Time: 1.6707048416137695s


2it [00:15,  7.50s/it]

...

48it [06:54,  7.67s/it]

mosaic-features Time: 0.0069010257720947266s
Qnodes to lookup: 2412
Qnodes from file: 2384
Outlier removal generates 3 lof-voted candidates
score-using-embedding Time: 1.0463471412658691s
compute-tf-idf-class_count Time: 1.5286352634429932s
compute-tf-idf-property_count Time: 1.7620868682861328s


49it [07:00,  7.17s/it]

mosaic-features Time: 0.007772922515869141s
Qnodes to lookup: 2904
Qnodes from file: 2835
Outlier removal generates 5 lof-voted candidates
score-using-embedding Time: 1.0382158756256104s
compute-tf-idf-class_count Time: 1.5172932147979736s
compute-tf-idf-property_count Time: 1.7567429542541504s


50it [07:07,  8.55s/it]


### Generate Training Data

In [ ]:
# features = [
#             "monge_elkan","monge_elkan_aliases","jaro_winkler",
#             "levenshtein","singleton","context_score_3",
#             "pgt_centroid_score","pgt_class_count_tf_idf_score",
#             "pgt_property_count_tf_idf_score", "num_tokens", "num_char"
#             ]

In [14]:
features = ["monge_elkan","monge_elkan_aliases","jaro_winkler",
            "levenshtein","singleton","context_score_3","pgt_centroid_score","pgt_class_count_tf_idf_score",
            "pgt_property_count_tf_idf_score", "num_occurences"]

In [15]:
def merge_files(args):
    datapath = args.train_path
    df_list  = []
    for fn in glob.glob(f"{datapath}/*csv"):
        if os.path.getsize(fn) == 0:
            continue
        fid = fn.split('/')[-1][:-4]
        df = pd.read_csv(fn)
        df['table_id'] = fid
        df['context_score'].fillna(0.0, inplace=True)
        df_list.append(df)            
    return pd.concat(df_list) 

def compute_normalization_factor(args, all_data):
    min_max_scaler_path = args.min_max_scaler_path
    all_data_features = all_data[features]
    scaler = MinMaxScaler()
    scaler.fit(all_data_features)
    pickle.dump(scaler, open(min_max_scaler_path, 'wb'))
    return scaler

In [16]:
def generate_train_data(args):
    scaler_path = args.min_max_scaler_path
    scaler = pickle.load(open(scaler_path, 'rb'))
    final_list = []
    sfeatures = copy.deepcopy(features) + ['evaluation_label']
    normalize_features = features
    evaluation_label = ['evaluation_label']
    positive_features_final = []
    negative_features_final = []
    for i,file in enumerate(glob.glob(args.train_path + '/*.csv')):
        file_name = file.split('/')[-1]
        if os.path.getsize(file) == 0:
                continue
        d_sample = pd.read_csv(file)
#         d_sample['context_score'].fillna(0.0, inplace=True)
        grouped_obj = d_sample.groupby(['column', 'row'])
        for cell in grouped_obj:
            cell[1][normalize_features] = scaler.transform(cell[1][normalize_features])
            pos_features = []
            neg_features = []
            a = cell[1][cell[1]['evaluation_label'] == 1]
            if a.empty:
                continue
            pos_rows = cell[1][cell[1]['evaluation_label'].astype(int) == 1][features].to_numpy()
            for i in range(len(pos_rows)):
                pos_features.append(pos_rows[i])
            neg_rows = cell[1][cell[1]['evaluation_label'].astype(int) == -1][features].to_numpy()
            for i in range(min(100,len(neg_rows))):
                neg_features.append(neg_rows[i])
            random.shuffle(pos_features)
            random.shuffle(neg_features)
            positive_features_final.append(pos_features)
            negative_features_final.append(neg_features)
            
    print(len(positive_features_final), len(positive_features_final[3]))
    print(len(negative_features_final), len(negative_features_final[3]))
    pickle.dump(positive_features_final,open(args.pos_output,'wb'))
    pickle.dump(negative_features_final,open(args.neg_output,'wb'))

In [17]:
gen_training_data_args = Namespace(train_path=train_feature_path, pos_output=pos_output, neg_output=neg_output, 
                 min_max_scaler_path=min_max_scaler_path)
all_data = merge_files(gen_training_data_args)
scaler = compute_normalization_factor(gen_training_data_args, all_data)

/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/1529062613.py:3: DtypeWarning: Columns (29,30,34,36) have mixed types.Specify dtype option on import or set low_memory=False.
  all_data = merge_files(gen_training_data_args)


In [18]:
len(all_data)

1986437

In [19]:
all_data.head()

,column,row,label,context,filename,column-id,label_clean,kg_id,kg_labels,kg_aliases,...,context_similarity,num_char,num_tokens,is_lof,pgt_centroid_score,pgt_class_count_tf_idf_score,top5_class_count,pgt_property_count_tf_idf_score,top5_property_count,table_id
0,0,0,Yuvileine,15830|84|2.83,042AKDN1.csv,042AKDN1.csv-0,Yuvileine,Q4533538,Yubileini (Krasnodar),03651430116|03251830004|Юбилейный,...,[],21,2,-1,0.586126,0.496094,Q56061:0.031|Q15642541:0.029|Q486972:0.028|Q17...,0.250704,P1082:0.062|P2044:0.050|P281:0.047|P1705:0.034...,042AKDN1
1,0,0,Yuvileine,15830|84|2.83,042AKDN1.csv,042AKDN1.csv-0,Yuvileine,Q24011352,Juviles,1125918951|2516238,...,[],7,1,-1,0.708974,0.406507,Q486972:0.028|Q177634:0.028|Q874405:0.027|Q788...,0.204015,P1376:0.061|P6766:0.054|P1566:0.031|P625:0.023...,042AKDN1
2,0,0,Yuvileine,15830|84|2.83,042AKDN1.csv,042AKDN1.csv-0,Yuvileine,Q951930,Juviles,/m/06t0bf|2516238|762927|1125918951|345896|911...,...,[],7,1,-1,0.603325,0.529422,Q12076836:0.032|Q56061:0.031|Q15642541:0.029|Q...,0.579808,P2046:0.064|P1082:0.062|P6766:0.054|P2044:0.05...,042AKDN1
3,0,0,Yuvileine,15830|84|2.83,042AKDN1.csv,042AKDN1.csv-0,Yuvileine,Q4533552,Mordovia Arena,away|rus/mordovia_arena|Yubileyniy Stadium|mor...,...,[],14,2,-1,0.502851,0.263346,Q1496967:0.026|Q82794:0.020|Q27096235:0.018|Q4...,0.106026,P373:0.029|P625:0.023|P131:0.021|P17:0.016|P18...,042AKDN1
4,0,0,Yuvileine,15830|84|2.83,042AKDN1.csv,042AKDN1.csv-0,Yuvileine,Q34716498,Yuvi Taando’o,Yuvi Taando'o|11220481,...,[],13,2,-1,0.580654,0.033954,Q618123:0.014|Q27096213:0.008|Q58416391:0.007|...,0.067667,P1566:0.031|P625:0.023|P17:0.016|P31:-0.003,042AKDN1


In [20]:
generate_train_data(gen_training_data_args)

/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3563378694.py:1: DtypeWarning: Columns (29,30,34,36) have mixed types.Specify dtype option on import or set low_memory=False.
  generate_train_data(gen_training_data_args)
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3563378694.py:1: DtypeWarning: Columns (29,30,34,36) have mixed types.Specify dtype option on import or set low_memory=False.
  generate_train_data(gen_training_data_args)


12671 1
12671 100


### Model Definition

In [21]:
# Dataset
class T2DV2Dataset(Dataset):
    def __init__(self, pos_features, neg_features):
        self.pos_features = pos_features
        self.neg_features = neg_features
    
    def __len__(self):
        return len(self.pos_features)
    
    def __getitem__(self, idx):
        return self.pos_features[idx], self.neg_features[idx]

# Model
class PairwiseNetwork(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        #original 12x24, 24x12, 12x12, 12x1
        self.fc1 = nn.Linear(hidden_size, 2*hidden_size)
        self.fc2 = nn.Linear(2*hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, 1)
    
    def forward(self, pos_features, neg_features):
        # Positive pass
        x = F.relu(self.fc1(pos_features))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        pos_out = torch.sigmoid(self.fc4(x))
        
        # Negative Pass
        x = F.relu(self.fc1(neg_features))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        neg_out = torch.sigmoid(self.fc4(x))
        
        return pos_out, neg_out
    
    def predict(self, test_feat):
        x = F.relu(self.fc1(test_feat))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        test_out = torch.sigmoid(self.fc4(x))
        return test_out

# Pairwise Loss
class PairwiseLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.m = 0
    
    def forward(self, pos_out, neg_out):
        distance = (1 - pos_out) + neg_out
        loss = torch.mean(torch.max(torch.tensor(0), distance))
        return loss

### Training

In [22]:
def generate_dataloader(positive_feat_path, negative_feat_path):
    pos_features = pickle.load(open(positive_feat_path, 'rb'))
    neg_features = pickle.load(open(negative_feat_path, 'rb'))

    pos_features_flatten = list(chain.from_iterable(pos_features))
    neg_features_flatten = list(chain.from_iterable(neg_features))

    train_dataset = T2DV2Dataset(pos_features_flatten, neg_features_flatten)
    train_dataloader = DataLoader(train_dataset, batch_size=64)
    return train_dataloader

def infer_scores(min_max_scaler_path, input_table_path, output_table_path, model):
    scaler = pickle.load(open(min_max_scaler_path, 'rb'))
    normalize_features = features
    for file in glob.glob(input_table_path + '/*.csv'):
        file_name = file.split('/')[-1]
        if os.path.getsize(file) == 0:
                continue
                
        d_sample = pd.read_csv(file)
#         d_sample['context_score'].fillna(0.0, inplace=True)
        grouped_obj = d_sample.groupby(['column', 'row'])
        new_df_list = []
        pred = []
        for cell in grouped_obj:
            cell[1][normalize_features] = scaler.transform(cell[1][normalize_features])
            sorted_df = cell[1].sort_values('context_score', ascending=False)
            sorted_df_features = sorted_df[normalize_features]
            new_df_list.append(sorted_df)
            arr = sorted_df_features.to_numpy()
            test_inp = []
            for a in arr:
                test_inp.append(a)
            test_tensor = torch.tensor(test_inp).float()
            scores = model.predict(test_tensor)
            scores_list = torch.squeeze(scores).tolist()
            if not type(scores_list) is list:
                pred.append(scores_list)
            else:
                pred.extend(scores_list)
        test_df = pd.concat(new_df_list)
        test_df[final_score_column] = pred
        test_df.to_csv(f"{output_table_path}/{file_name}", index=False)

def train(args):
    if torch.cuda.is_available():
        device = torch.device('cuda')
    
    else:
        device = torch.device('cpu')
    train_dataloader = generate_dataloader(args.positive_feat_path, args.negative_feat_path)
    criterion = PairwiseLoss()
    EPOCHS = args.num_epochs
    model = PairwiseNetwork(len(features)).to(device=device)
    optimizer = Adam(model.parameters(), lr=args.lr)
    top1_max_prec = 0
    for epoch in range(EPOCHS):
        train_epoch_loss = 0
        avg_loss = 0
        model.train()
        for bid, batch in tqdm(enumerate(train_dataloader), position=0, leave=True):
            positive_feat = torch.tensor(batch[0].float())
            negative_feat = torch.tensor(batch[1].float())
            optimizer.zero_grad()
            pos_out, neg_out = model(positive_feat, negative_feat)
            loss = criterion(pos_out, neg_out)
            loss.backward()
            optimizer.step()
            train_epoch_loss += loss
        avg_loss = train_epoch_loss / bid

        # Evaluation
        model.eval()
        infer_scores(args.min_max_scaler_path, args.dev_path, args.dev_output, model)
        eval_data = merge_eval_files(args.dev_output)
        res, candidate_eval_data = parse_eval_files_stats(eval_data, final_score_column)
        top1_precision = res['num_tasks_with_model_score_top_one_accurate']/res['num_tasks_with_gt']
        if top1_precision > top1_max_prec:
            top1_max_prec = top1_precision
            model_save_name = 'epoch_{}_loss_{}_top1_{}.pth'.format(epoch, avg_loss, top1_max_prec)
            best_model_path = os.path.join(args.model_save_path, model_save_name)
            torch.save(model.state_dict(), best_model_path)
        
        print("Epoch {}, Avg Loss is {}, epoch top1 {}, max top1 {}".format(epoch, avg_loss, top1_precision, top1_max_prec))
    return best_model_path

In [23]:
def merge_eval_files(final_score_path):
    eval_file_names = []
    df_list = []
    for (dirpath, dirnames, filenames) in os.walk(final_score_path):
        for fn in filenames:
            if "csv" not in fn:
                continue
            abs_fn = os.path.join(dirpath, fn)
            assert os.path.isfile(abs_fn)
            if os.path.getsize(abs_fn) == 0:
                continue
            eval_file_names.append(abs_fn)
    
    for fn in eval_file_names:
        fid = fn.split('/')[-1].split('.csv')[0]
        df = pd.read_csv(fn)
        df['table_id'] = fid
        df_list.append(df)
    return pd.concat(df_list)

def parse_eval_files_stats(eval_data, method):
    res = {}
    candidate_eval_data = eval_data.groupby(['table_id', 'column', 'row'])['table_id'].count().reset_index(name="count")
    res['num_tasks_with_gt'] = len(eval_data[pd.notna(eval_data['GT_kg_id'])].groupby(['table_id', 'column', 'row']))
    num_tasks_with_model_score_top_one_accurate = []
    num_tasks_with_model_score_top_five_accurate = []
    num_tasks_with_model_score_top_ten_accurate = []
    has_gt_list = []
    has_gt_in_candidate = []
    for i, row in candidate_eval_data.iterrows():
        table_id, row_idx, col_idx = row['table_id'], row['row'], row['column']
        c_e_data = eval_data[(eval_data['table_id'] == table_id) & (eval_data['row'] == row_idx) & (eval_data['column'] == col_idx)]
        assert len(c_e_data) > 0
        if np.nan not in set(c_e_data['GT_kg_id']):
            has_gt_list.append(1)
        else:
            has_gt_list.append(0)
        if 1 in set(c_e_data['evaluation_label']):
            has_gt_in_candidate.append(1)
        else:
            has_gt_in_candidate.append(0)
                    
        #rank on model score
        s_data = c_e_data.sort_values(by=[method], ascending=False)
        if s_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_model_score_top_one_accurate.append(1)
        else:
            num_tasks_with_model_score_top_one_accurate.append(0)
        if 1 in set(s_data.iloc[0:5]['evaluation_label']):
            num_tasks_with_model_score_top_five_accurate.append(1)
        else:
            num_tasks_with_model_score_top_five_accurate.append(0)
        if 1 in set(s_data.iloc[0:10]['evaluation_label']):
            num_tasks_with_model_score_top_ten_accurate.append(1)
        else:
            num_tasks_with_model_score_top_ten_accurate.append(0)
            
    res['num_tasks_with_model_score_top_one_accurate'] = sum(num_tasks_with_model_score_top_one_accurate)
    res['num_tasks_with_model_score_top_five_accurate'] = sum(num_tasks_with_model_score_top_five_accurate)
    res['num_tasks_with_model_score_top_ten_accurate'] = sum(num_tasks_with_model_score_top_ten_accurate)
    return res, candidate_eval_data

In [24]:
training_args = Namespace(num_epochs=20, lr=0.001, positive_feat_path=pos_output, negative_feat_path=neg_output,
                         dev_path=dev_feature_path, dev_output=dev_output_predictions,
                         model_save_path=model_save_path, min_max_scaler_path=min_max_scaler_path)

In [25]:
## Call Training
best_model_path = train(training_args)

0it [00:00, ?it/s]/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3986128095.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_feat = torch.tensor(batch[0].float())
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3986128095.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negative_feat = torch.tensor(batch[1].float())
198it [00:00, 549.31it/s]
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3986128095.py:74: DtypeWarning: Columns (29,30,34,36) have mixed types.Specify dtype option on import or set low_memory=False.
  infer_scores(args.min_max_scaler_path, args.dev_path, args.dev_output, model)
/var/folders/qv/cxzpwz3j29x7n79

Epoch 0, Avg Loss is 0.6426814794540405, epoch top1 0.9640381282495667, max top1 0.9640381282495667


0it [00:00, ?it/s]/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3986128095.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_feat = torch.tensor(batch[0].float())
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3986128095.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negative_feat = torch.tensor(batch[1].float())
198it [00:00, 532.73it/s]
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3986128095.py:74: DtypeWarning: Columns (29,30,34,36) have mixed types.Specify dtype option on import or set low_memory=False.
  infer_scores(args.min_max_scaler_path, args.dev_path, args.dev_output, model)
/var/folders/qv/cxzpwz3j29x7n79

Epoch 1, Avg Loss is 0.04851575940847397, epoch top1 0.9649046793760832, max top1 0.9649046793760832


0it [00:00, ?it/s]/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3986128095.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_feat = torch.tensor(batch[0].float())
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3986128095.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negative_feat = torch.tensor(batch[1].float())
198it [00:00, 520.71it/s]
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3986128095.py:74: DtypeWarning: Columns (29,30,34,36) have mixed types.Specify dtype option on import or set low_memory=False.
  infer_scores(args.min_max_scaler_path, args.dev_path, args.dev_output, model)
/var/folders/qv/cxzpwz3j29x7n79

...

Epoch 17, Avg Loss is 0.014523069374263287, epoch top1 0.9675043327556326, max top1 0.9675043327556326


0it [00:00, ?it/s]/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3986128095.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_feat = torch.tensor(batch[0].float())
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3986128095.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negative_feat = torch.tensor(batch[1].float())
198it [00:00, 475.37it/s]
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3986128095.py:74: DtypeWarning: Columns (29,30,34,36) have mixed types.Specify dtype option on import or set low_memory=False.
  infer_scores(args.min_max_scaler_path, args.dev_path, args.dev_output, model)
/var/folders/qv/cxzpwz3j29x7n79

Epoch 18, Avg Loss is 0.014347709715366364, epoch top1 0.9666377816291161, max top1 0.9675043327556326


0it [00:00, ?it/s]/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3986128095.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_feat = torch.tensor(batch[0].float())
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3986128095.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negative_feat = torch.tensor(batch[1].float())
198it [00:00, 474.53it/s]
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_3197/3986128095.py:74: DtypeWarning: Columns (29,30,34,36) have mixed types.Specify dtype option on import or set low_memory=False.
  infer_scores(args.min_max_scaler_path, args.dev_path, args.dev_output, model)
/var/folders/qv/cxzpwz3j29x7n79

Epoch 19, Avg Loss is 0.0142440190538764, epoch top1 0.9670710571923743, max top1 0.9675043327556326


In [26]:
best_model_path

'/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/table-linker/dev-output/tbl-pipeline-nn/v26/saved_models/epoch_17_loss_0.014523069374263287_top1_0.9675043327556326.pth'

## Dev Prediction

In [27]:
def dev_prediction(dev_feature_path, dev_predictions_top_k, saved_model, output_column, min_max_scaler_path, k=5):
    for file in glob.glob(dev_feature_path + '/*.csv'):
        filename = file.split("/")[-1]
        print(filename)
        feature_str =  ",".join(features)
        if os.path.getsize(file) == 0:
            continue
        # location where the output generated by the predictions wil be stored.
        dev_output = f"{dev_predictions_top_k}/{filename}"
        !tl --log-file $tl_log_file predict-using-model $file -o $output_column \
            --features $feature_str \
            --ranking-model $saved_model \
            --normalization-factor $min_max_scaler_path \
            / get-kg-links -c $final_score_column -k $k --k-rows \
            > $dev_output

In [28]:
def add_color(dev_predictions_top_k, dev_colorized_path, score_column, k=5):
    for file in glob.glob(dev_predictions_top_k + '/*.csv'):
        filename = file.split("/")[-1]
        print(filename)
        if os.path.getsize(file) == 0:
                    continue
                
        dev_color_file = f"{dev_colorized_path}/{filename[:-4]}.xlsx"
        !tl add-color $file -c "$score_column,evaluation_label" -k $k --output $dev_color_file

In [29]:
def compute_metrics(dev_predictions_top_k, dev_predictions_metrics, score_column, k=5):
    df_list = []
    for file in glob.glob(dev_predictions_top_k + '/*.csv'):
        filename = file.split("/")[-1]
        if os.path.getsize(file) == 0:
                    continue
                
        dev_metrics_file = f"{dev_predictions_metrics}/{filename}"
        !tl --log-file $tl_log_file metrics $file -k $k -c $score_column --tag $filename> $dev_metrics_file
        df_list.append(pd.read_csv(dev_metrics_file))
    return pd.concat(df_list)

In [30]:
def compute_custom_metrics(dev_predictions_top_k):
    df_list = []
    for file in glob.glob(dev_predictions_top_k+"/*.csv"):
        filename = file.split("/")[-1]
        print(filename)
        if os.path.getsize(file) == 0:
                    continue
        df = pd.read_csv(file)
        col_df = []
        for col, coldf in df.groupby(by=["column"]):
            rows = 0
            pgt_rows = 0
            pgt_recall = 0
            unignored_rows = 0
            unignored_candidates = 0
            unignored_correct = 0
            ignored_correct = 0
            kth_perc_rows = 0
            kth_perc_correct = 0
            kth_perc_candidates = 0
            for row, rowdf in coldf.groupby(by=["row"]):
                rows += 1
                p_count = rowdf[(rowdf["pseudo_gt"] == 1)].shape[0]
                if p_count > 0:
                    pgt_rows += 1
                p_recall = rowdf[((rowdf["pseudo_gt"] == 1) & (rowdf["evaluation_label"] == 1))].shape[0]
                pgt_recall += p_recall
                ignore_0_count = rowdf[rowdf["ignore_candidate"] == 0].shape[0]
                if ignore_0_count > 0:
                    unignored_rows += 1
                unignored_candidates += ignore_0_count
                unignored_correct += rowdf[((rowdf["ignore_candidate"] == 0) & (rowdf["evaluation_label"] == 1))].shape[0]
                ignored_correct += rowdf[((rowdf["ignore_candidate"] == 1) & (rowdf["evaluation_label"] == 1))].shape[0]
                kth_perc_1_count = rowdf[rowdf["kth_percenter"] == 1].shape[0]
                if kth_perc_1_count > 0:
                    kth_perc_rows += 1
                kth_perc_candidates += kth_perc_1_count
                kth_perc_correct += rowdf[((rowdf["kth_percenter"] == 1) & (rowdf["evaluation_label"] == 1))].shape[0]
            unique_rows = coldf["label"].nunique()
            col_df.append(pd.DataFrame([{
                "filename":filename,
                "column": col,
                "rows": rows,
                "unique_rows": unique_rows,
                "pgt_rows": pgt_rows,
                "pgt_recall": pgt_recall,
                "pgt_accuracy": pgt_recall/pgt_rows if pgt_rows!=0 else 0,
                "unignored_rows": unignored_rows,
                "unignored_candidates": unignored_candidates,
                "unignored_correct": unignored_correct,
                "ignored_correct": ignored_correct,
                "ignore_candidate_accuracy": unignored_correct/unignored_rows if unignored_rows != 0 else 0,
                "kth_percenter_rows": kth_perc_rows,
                "kth_percenter_candidates": kth_perc_candidates,
                "kth_percenter_correct": kth_perc_correct,
                "kth_percenter_accuracy": kth_perc_correct/kth_perc_rows
            }]))
        df_list.append(pd.concat(col_df))
    return pd.concat(df_list)

In [61]:
min_max_scaler_path

'/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/table-linker/dev-output/tbl-pipeline-nn/v26/training_data/tl_pipeline_normalization_factor.pkl'

In [31]:
dev_prediction(dev_feature_path, dev_predictions_top_k, best_model_path, final_score_column, min_max_scaler_path, k=5)

4PKEEJU4.csv
4DC5O5I4.csv
58E7A5WL.csv
0FQOOJPU.csv
5IXA0RAI.csv
0LF7RI6N.csv
2PKG4E2V.csv
50NWQJ1T.csv
44NDHWR1.csv
39W7XXTI.csv
3WXFYEAX.csv
28D7RBJT.csv
1GUVMENF.csv
0QWF60VG.csv
3OCW1LDZ.csv
4V4O0CTS.csv
29D1VZHF.csv
4FG1UN8O.csv
0LZ0M8W4.csv
3B54GZSX.csv
1ZFRQBQS.csv
0WBTX8LY.csv
1NS33P8C.csv
2TGNKH1P.csv
53OUTCE4.csv
5TJI4XTK.csv
4DPRZWVL.csv
080HU8A5.csv
3OAZEVOY.csv
13BLTPJD.csv
3M5QXPWN.csv
0G9YPQC0.csv
1YPLVLS9.csv
0CETTKTA.csv
1GOKLC0K.csv
2QFYH2N9.csv
00ECUL14.csv
2E6QBLCA.csv
1T91CHXV.csv
59W76Q0Y.csv
5PKTGQ6Q.csv
3JNFST2K.csv
1XIWQBSF.csv
2YCSL7OH.csv
2FSRG0OI.csv
0P8H49LQ.csv
2FXR6BX7.csv
1PTL0CX1.csv
0TQXSY28.csv
0JSF530F.csv


In [38]:
metrics_df = compute_metrics(dev_predictions_top_k, dev_metrics_path, final_score_column, k=5)

In [39]:
metrics_df

,k,f1,precision,recall,column,tag
0,5,1.000000,1.000000,1.0,0,4PKEEJU4.csv
1,5,1.000000,1.000000,1.0,1,4PKEEJU4.csv
2,5,1.000000,1.000000,1.0,2,4PKEEJU4.csv
0,5,1.000000,1.000000,1.0,0,4DC5O5I4.csv
0,5,0.926829,0.863636,1.0,0,58E7A5WL.csv
...,...,...,...,...,...,...
3,5,1.000000,1.000000,1.0,4,2FXR6BX7.csv
0,5,1.000000,1.000000,1.0,0,1PTL0CX1.csv
1,5,0.974359,0.950000,1.0,2,1PTL0CX1.csv
0,5,1.000000,1.000000,1.0,0,0TQXSY28.csv


In [44]:
metrics_df.to_csv(f"{dev_metrics_path}/metrics_5.csv", index=False)

In [41]:
metrics_df['f1'].mean()

0.9753624297138684

In [42]:
metrics_df['precision'].mean()

0.9689103876603876

In [43]:
metrics_df['recall'].mean()

0.9921921921921921

In [45]:
add_color(dev_predictions_top_k, dev_colorized_path, final_score_column)

4PKEEJU4.csv
add-color Time: 0.15696001052856445s
4DC5O5I4.csv
add-color Time: 0.08216977119445801s
58E7A5WL.csv
add-color Time: 0.1220247745513916s
0FQOOJPU.csv
add-color Time: 0.1384727954864502s
5IXA0RAI.csv
add-color Time: 0.08520293235778809s
0LF7RI6N.csv
add-color Time: 0.11289191246032715s
2PKG4E2V.csv
add-color Time: 0.17091608047485352s
50NWQJ1T.csv
add-color Time: 0.1406090259552002s
44NDHWR1.csv
add-color Time: 0.16513991355895996s
39W7XXTI.csv
add-color Time: 0.13980412483215332s
3WXFYEAX.csv
add-color Time: 0.07999324798583984s
28D7RBJT.csv
add-color Time: 0.11517214775085449s
1GUVMENF.csv
add-color Time: 0.08106207847595215s
0QWF60VG.csv
add-color Time: 0.15035700798034668s
3OCW1LDZ.csv
add-color Time: 0.13845205307006836s
4V4O0CTS.csv
add-color Time: 0.11144614219665527s
29D1VZHF.csv
add-color Time: 0.14304208755493164s
4FG1UN8O.csv
add-color Time: 0.1822950839996338s
0LZ0M8W4.csv
add-color Time: 0.15276432037353516s
3B54GZSX.csv
add-color Time: 0.14145374298095703s
1ZFR

In [46]:
def find_missing_correct_candidates(candidates_path, missing_correct_candidates_path):
     for file in tqdm(glob.glob(candidates_path + '/*.csv')):
        filename = file.split("/")[-1]
        if os.path.getsize(file) == 0:
                    continue
        missing_file = f"{missing_correct_candidates_path}/{filename}"
        !tl check-candidates "$file" > "$missing_file"

In [47]:
find_missing_correct_candidates(dev_candidate_path, dev_missing_candidates_path)

  0%|                                                                                             | 0/51 [00:00<?, ?it/s]

check-candidates Time: 0.015120744705200195s


  2%|█▋                                                                                   | 1/51 [00:02<01:41,  2.04s/it]

check-candidates Time: 0.00684809684753418s


  4%|███▎                                                                                 | 2/51 [00:03<01:37,  1.98s/it]

...

 96%|████████████████████████████████████████████████████████████████████████████████▋   | 49/51 [01:36<00:04,  2.05s/it]

check-candidates Time: 0.006526947021484375s


 98%|██████████████████████████████████████████████████████████████████████████████████▎ | 50/51 [01:38<00:02,  2.03s/it]

check-candidates Time: 0.0066852569580078125s


100%|████████████████████████████████████████████████████████████████████████████████████| 51/51 [01:40<00:00,  1.97s/it]


In [48]:
def concat_files(files_path):
    df_list = []
    for file in glob.glob(files_path + '/*.csv'):
        filename = file.split("/")[-1]
        print(filename)
        if os.path.getsize(file) == 0:
                    continue
        df = pd.read_csv(file)
        df['filename'] = filename
        df_list.append(df)
    return pd.concat(df_list)

In [49]:
missing_df = concat_files(dev_missing_candidates_path)

4PKEEJU4.csv
4DC5O5I4.csv
58E7A5WL.csv
0FQOOJPU.csv
5IXA0RAI.csv
0LF7RI6N.csv
2PKG4E2V.csv
50NWQJ1T.csv
44NDHWR1.csv
39W7XXTI.csv
3WXFYEAX.csv
28D7RBJT.csv
1GUVMENF.csv
0QWF60VG.csv
3OCW1LDZ.csv
4V4O0CTS.csv
29D1VZHF.csv
4FG1UN8O.csv
0LZ0M8W4.csv
3B54GZSX.csv
1ZFRQBQS.csv
0WBTX8LY.csv
1NS33P8C.csv
2TGNKH1P.csv
53OUTCE4.csv
5TJI4XTK.csv
4DPRZWVL.csv
080HU8A5.csv
3OAZEVOY.csv
13BLTPJD.csv
3M5QXPWN.csv
0G9YPQC0.csv
1YPLVLS9.csv
0CETTKTA.csv
1GOKLC0K.csv
2QFYH2N9.csv
00ECUL14.csv
2E6QBLCA.csv
1T91CHXV.csv
59W76Q0Y.csv
5PKTGQ6Q.csv
3JNFST2K.csv
1XIWQBSF.csv
2YCSL7OH.csv
2FSRG0OI.csv
0P8H49LQ.csv
2FXR6BX7.csv
1PTL0CX1.csv
0TQXSY28.csv
0JSF530F.csv


In [50]:
len(missing_df)

18

In [51]:
missing_df.to_csv(f"{dev_missing_candidates_path}/missing_concatenated.csv", index=False)

In [52]:
find_missing_correct_candidates(train_candidate_path, train_missing_candidates_path)

  0%|                                                                                            | 0/301 [00:00<?, ?it/s]

check-candidates Time: 0.006678104400634766s


  0%|▎                                                                                   | 1/301 [00:01<09:58,  1.99s/it]

check-candidates Time: 0.00661778450012207s


  1%|▌                                                                                   | 2/301 [00:03<09:56,  1.99s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████████████▍| 299/301 [10:43<00:04,  2.31s/it]

check-candidates Time: 0.0070879459381103516s


100%|█████████████████████████████████████████████████████████████████████████████████▋| 300/301 [10:47<00:02,  2.73s/it]

check-candidates Time: 0.01830601692199707s


100%|██████████████████████████████████████████████████████████████████████████████████| 301/301 [10:50<00:00,  2.16s/it]


In [53]:
train_missing_df = concat_files(train_missing_candidates_path)

042AKDN1.csv
1DQVRVUL.csv
2VWPPCQM.csv
4IJHCIR9.csv
2OYAB7YM.csv
5894JWA5.csv
4Q6FAT24.csv
3S6WON79.csv
4IB27COT.csv
2S72D3R9.csv
49K301SA.csv
03XJOJ15.csv
0KL64BZL.csv
1XV2X3XE.csv
2J3MZCRD.csv
59CFTDPL.csv
19WCLMKZ.csv
4722C9EQ.csv
59JNC864.csv
4KDMZHXJ.csv
17DA0MJ4.csv
2WRDGDBJ.csv
4UUFDEP4.csv
0VDX10IJ.csv
1QGWWYAF.csv
0IUSS6UH.csv
42BKS5LP.csv
40DGQ0R6.csv
21P58N2X.csv
10WA15QW.csv
4ENVC6F0.csv
39DC3Z7N.csv
302H92MV.csv
5VD9H3AD.csv
1AXBOBYJ.csv
4VL4BFPM.csv
0Z8LKW3C.csv
0KWK2FSC.csv
28Y667AR.csv
3K0JGMP1.csv
1A51CWBN.csv
4UZA5UWB.csv
15CM535P.csv
3CSAH6ZY.csv
0MYK9SQC.csv
06UFHM3B.csv
06ZS1QBE.csv
14DACNEL.csv
2OJXQ9F3.csv
0HZG8QGJ.csv
5TWQ98LE.csv
24OGNVVP.csv
0XWQ7DA6.csv
0ADP8WGU.csv
0P4Y07PN.csv
4C37XZHJ.csv
26CIDK81.csv
5J25NATF.csv
3AK02Q9B.csv
3EL4J49W.csv
1JY71MSY.csv
4LFJR3YN.csv
243JI1XC.csv
3SQ9UAK1.csv
258AFYQJ.csv
3HZEPYNC.csv
0ZZFENJS.csv
1XK9O8CF.csv
20PYU63R.csv
471N0AUD.csv
03Q5JG9I.csv
4N6X42PJ.csv
2FHCJAMA.csv
2ZYEX9U9.csv
4E99I3P7.csv
0VGKU21T.csv
4VAYQOG0.csv

In [54]:
len(train_missing_df)

106

In [55]:
train_missing_df

,column,row,label,context,GT_kg_id,GT_kg_label,filename
0,0,3,JD 2194,-46.207207|-0.16881000000000002,Q12067892,HD 23194,5894JWA5.csv
1,0,7,JD 22234,61.28388|-0.88644,Q12067734,HD 222349,5894JWA5.csv
0,0,0,[FK2004] 118,44.74431|0.024648029999999998|160.17874999999998,Q78536288,[FK2002] 118,2J3MZCRD.csv
1,0,5,"[FK2002]""167",58.805476899999995|0.015226542999999999|186.49...,Q77924250,[FK2002] 167,2J3MZCRD.csv
2,0,6,[FK2002]25,56.67630912|0.00647559|165.01396464,Q77907580,[FK2002] 125,2J3MZCRD.csv
...,...,...,...,...,...,...,...
2,2,40,Kassrine,Arch of Septimis Severvs|listed monument of Tu...,Q276576,Beja Governorate,0VC79Y6X.csv
3,2,41,Kassrine,Arch of Alxander Severus|listed monument of Tu...,Q276576,Beja Governorate,0VC79Y6X.csv
0,0,2,John,House of Plantagenet|John Lackland|Beaumont Pa...,Q129308,"John, King of England",3IB64ASX.csv
0,1,12,Turkestan,Turkistan|Brockhaus and Efron Encyclopedic Dic...,Q6549315,Turkestan,471ODBDH.csv


In [56]:
train_missing_df.to_csv(f"{train_missing_candidates_path}/missing_concatenated.csv", index=False)

In [57]:
def analysis(predictions_top_k):
    analysis = []
    for file in glob.glob(predictions_top_k + '/*.csv'):
        filename = file.split("/")[-1]
        if os.path.getsize(file) == 0:
                    continue
        df = pd.read_csv(file)
        df.fillna(0.0, inplace=True)
        col_grpd = df.groupby(by=['column'])
        for col, cgdf in col_grpd:
            cs_avg = cgdf[cgdf['evaluation_label'] == 1]['context_score_3'].mean()
            pgt_centroid_score_avg = cgdf[cgdf['evaluation_label'] == 1]['pgt_centroid_score'].mean()
            pgt_class_count_tf_idf_avg = cgdf[cgdf['evaluation_label'] == 1]['pgt_class_count_tf_idf_score'].mean()
            pgt_prop_count_tf_idf_avg = cgdf[cgdf['evaluation_label'] == 1]['pgt_property_count_tf_idf_score'].mean()
            row_grpd = cgdf.groupby(by=['row'])
            for row, rgdf in row_grpd:
                rgdf = rgdf[rgdf['rank'].astype(int) == 1]
                if filename == '0QWF60VG.csv' and col == 0 and row == 0:
                    print(rgdf['kg_id'], rgdf['GT_kg_id'], rgdf['evaluation_label'])
                for i, r in rgdf.iterrows():
                    if r['evaluation_label'] == -1 and r['kg_id'] != r['GT_kg_id']:
                        analysis.append({
                            'filename': filename,
                            'column': col,
                            'row': row,
                            'kg_id': r['kg_id'],
                            'label': r['label'],
                            'GT_kg_id': r['GT_kg_id'],
                            'GT_kg_label': r['GT_kg_label'],
                            'context_score_3_average': cs_avg,
                            'context_score_3': r['context_score_3'],
                            'pgt_centroid_score_avg': pgt_centroid_score_avg,
                            'pgt_centroid_score': r['pgt_centroid_score'],
                            'pgt_class_count_tf_idf_avg': pgt_class_count_tf_idf_avg,
                            'pgt_class_count_tf_idf_score': r['pgt_class_count_tf_idf_score'],
                            'pgt_property_count_tf_idf_avg': pgt_prop_count_tf_idf_avg,
                            'pgt_property_count_tf_idf_score': r['pgt_property_count_tf_idf_score'],
                            'siamese_prediction': r['siamese_prediction'],
                            'best_str_similarity': r['best_str_similarity'],
                            'ignore_candidate': r['ignore_candidate'],
                            'kth_percenter': r['kth_percenter'],
                            'pseudo_gt': r['pseudo_gt']
                        })
    return pd.DataFrame(analysis).sort_values(by=['filename', 'column', 'row'], ascending=[True, True, True])

In [58]:
analysis_df = analysis(dev_predictions_top_k)

0    Q2322279
Name: kg_id, dtype: object 0    Q17606582
Name: GT_kg_id, dtype: object 0   -1
Name: evaluation_label, dtype: int64


In [59]:
analysis_df

,filename,column,row,kg_id,label,GT_kg_id,GT_kg_label,context_score_3_average,context_score_3,pgt_centroid_score_avg,pgt_centroid_score,pgt_class_count_tf_idf_avg,pgt_class_count_tf_idf_score,pgt_property_count_tf_idf_avg,pgt_property_count_tf_idf_score,siamese_prediction,best_str_similarity,ignore_candidate,kth_percenter,pseudo_gt
46,0G9YPQC0.csv,0,12,Q4718151,Alexander,Q1816386,Alexander,0.507937,0.000000,0.915846,0.181738,0.980586,0.026926,0.755590,0.106657,0.000063,1.000000,1,0,-1
47,0G9YPQC0.csv,2,0,Q1105359,cocktail,Q134768,cocktail,0.000000,0.000000,0.454861,1.000000,0.089061,1.000000,0.071663,0.931981,0.999977,1.000000,0,1,1
48,0G9YPQC0.csv,2,1,Q1105359,cocktail,Q134768,cocktail,0.000000,0.000000,0.454861,1.000000,0.089061,1.000000,0.071663,0.931981,0.999977,1.000000,1,0,-1
49,0G9YPQC0.csv,2,2,Q1105359,cocktail,Q134768,cocktail,0.000000,0.000000,0.454861,1.000000,0.089061,1.000000,0.071663,0.931981,0.999977,1.000000,1,0,-1
50,0G9YPQC0.csv,2,3,Q1105359,cocktail,Q134768,cocktail,0.000000,0.000000,0.454861,1.000000,0.089061,1.000000,0.071663,0.931981,0.999977,1.000000,1,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,58E7A5WL.csv,0,20,Q22246147,The Crypt,Q22246155,The Crypt,0.731661,0.507937,0.952374,0.994435,1.000000,1.000000,0.816708,0.844250,0.999968,1.000000,1,0,-1
4,58E7A5WL.csv,0,21,Q22245990,The Shadow on the House,Q22245982,The Shadow on the House,0.731661,0.507937,0.952374,0.992174,1.000000,1.000000,0.816708,0.844250,0.999968,1.000000,0,1,-1
71,5PKTGQ6Q.csv,0,5,Q64009286,2015 Vall-llobrega Town Council electjon,Q64512660,2015 Vall-llobrega Town Council election,0.519234,0.496428,0.926001,0.887802,1.000000,1.000000,0.931981,0.931981,0.999944,0.904414,1,0,-1
72,5PKTGQ6Q.csv,0,8,Q64492293,2011 Ullastret Town Councl election,Q64506148,2011 Ullastret Town Council election,0.519234,0.490891,0.926001,0.910435,1.000000,1.000000,0.931981,0.931981,0.999945,0.902434,1,0,-1


In [60]:
analysis_df.to_csv(dev_incorrect_answers_path, index=False)